# Word counts
A [`CountVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) using `SciKit-Learn` is used for extracting features from the data. The features are then used to train a **random forest classifier**.

Submission accuracy: `0.60492`

Download necessary NLTK files:
* `stopwords`: Stopwords Corpus
* `wordnet`: WordNet

In [1]:
%%script false
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

In [1]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from os.path import join
from tqdm import tqdm

In [2]:
src= 'data'
df_train = pd.read_csv(join(src, 'labeledTrainData.tsv'), sep='\t')
df_train.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


## Data cleaning
Here we use the data cleaning steps in `1_data_cleaning.ipynb`.

We also add the cleaned reviews to the `DataFrame` and save them in a new CSV for future use.

In [3]:
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))

def clean_sentence(sentence):
    removed_markup = BeautifulSoup(sentence, 'html.parser').get_text()
    removed_punctuation = re.sub(r'[^a-zA-Z]', ' ', removed_markup)
    tokens = removed_punctuation.lower().split()
    removed_stopwords = [w for w in tokens if w not in stopwords]
    lemmatized = [lemmatizer.lemmatize(w) for w in removed_stopwords]
    return ' '.join(lemmatized)

In [4]:
clean_sentence(df_train['review'][0])

'stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighty maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle message mj feeling towards press also obvious message drug bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fan would say made fan true really nice actual feature film bit finally start minute excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord want mj dead bad beyond mj overheard plan nah joe pesci character ranted wanted people know supplying drug etc dunno maybe hate mj music lot cool thing like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually director hate working one kid let a

In [5]:
df_train['review_cleaned'] = [clean_sentence(s) for s in tqdm(df_train['review'])]
df_train.head()

100%|██████████████████████████████████████████████████████████████████████████| 25000/25000 [00:24<00:00, 1020.48it/s]


,id,sentiment,review,review_cleaned
0,5814_8,1,With all this stuff going down at the moment w...,stuff going moment mj started listening music ...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...",classic war world timothy hines entertaining f...
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,film start manager nicholas bell giving welcom...
3,3630_4,0,It must be assumed that those who praised this...,must assumed praised film greatest filmed oper...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy wondrously unpretentious explo...


## Feature extraction
Extract features from the word counts. The features are the top-`n` words, with their corresponding counts in the data.

In [7]:
vectorizer = CountVectorizer(analyzer='word', max_features=5000)
train_features = vectorizer.fit_transform(df_train['review_cleaned'])
train_features = train_features.toarray()  # from scipy.sparse.csr.csr_matrix
train_features.shape

(25000, 5000)

Examine the word counts for our vocabulary.

In [9]:
vocab = vectorizer.get_feature_names()
dist = np.sum(train_features, axis=0)
for tag, count in zip(vocab, dist):
    print(tag, count)

abandoned 187
abc 125
ability 562
able 1259
abraham 92
abrupt 73
absence 118
absent 83
absolute 352
absolutely 1485
absurd 306
absurdity 86
abuse 202
abused 77
abusive 91
abysmal 98
academy 298
accent 688
accept 300
acceptable 130
accepted 144
accepts 74
access 92
accident 337
accidentally 200
acclaimed 75
accompanied 88
accomplish 77
accomplished 124
according 296
account 243
accuracy 82
accurate 284
accused 123
ace 73
achieve 179
achieved 139
achievement 169
acid 91
across 971
act 1645
acted 658
acting 6491
action 3665
active 76
activity 146
actor 6875
actress 1588
actual 793
actually 4237
ad 188
adam 400
adaptation 533
adapted 154
add 1147
added 439
addict 82
addicted 78
addiction 72
adding 166
addition 371
additional 74
address 103
adequate 113
admirable 73
admire 124
admit 621
admittedly 134
adolescent 94
adopted 75
adorable 101
adult 886
advance 140
advanced 90
advantage 170
adventure 714
advertising 91
advice 262
advise 90
affair 417
affect 155
affected 113
affection 105
afford 

emotionally 241
empathy 84
emperor 97
emphasis 105
empire 127
employee 94
empty 275
en 91
encounter 315
encourage 75
end 6632
endearing 139
ended 556
ending 2462
endless 235
endure 99
enemy 307
energy 324
engage 93
engaged 110
engaging 312
england 306
english 986
enjoy 1812
enjoyable 842
enjoyed 1245
enjoying 162
enjoyment 150
enjoys 112
enormous 103
enough 3452
ensemble 152
ensues 88
enter 196
enterprise 107
enters 131
entertain 172
entertained 237
entertaining 1442
entertainment 887
enthusiasm 86
entire 1460
entirely 532
entry 191
environment 196
epic 366
episode 2597
equal 176
equally 432
equipment 84
equivalent 89
er 89
era 642
eric 265
erotic 201
error 171
escape 701
escaped 110
especially 2535
essence 138
essential 168
essentially 257
established 164
estate 136
esther 75
et 103
etc 1212
eternal 72
ethan 92
ethnic 75
eugene 83
europe 216
european 340
eva 140
eve 119
even 12646
evening 259
event 1283
eventually 720
ever 5995
every 3978
everybody 411
everyday 167
everyone 2222
every

pleasantly 124
please 1045
pleased 127
pleasure 347
plenty 632
plight 94
plot 6870
plus 651
pocket 77
poe 74
poem 94
poetic 96
poetry 92
poignant 156
point 4039
pointed 135
pointless 504
poison 83
pokemon 93
polanski 106
pole 83
police 1098
policeman 117
policy 78
polished 85
political 608
politically 106
politician 118
politics 208
pool 158
poor 1897
poorly 713
pop 405
popcorn 112
popular 550
popularity 83
population 120
porn 368
porno 99
portion 143
portrait 164
portray 264
portrayal 578
portrayed 601
portraying 227
portrays 229
pose 101
posey 76
position 211
positive 548
positively 74
posse 136
possessed 108
possibility 205
possible 999
possibly 709
post 503
poster 196
pot 102
potential 616
potentially 91
pound 127
poverty 133
powell 214
power 1265
powerful 620
practically 234
practice 134
praise 196
praised 72
prank 76
pre 309
precious 121
precisely 77
predator 72
predecessor 104
predictable 854
prefer 175
pregnant 178
prejudice 108
premiere 83
preminger 73
premise 746
prepare 77
p

## Training
Use the features obtained from the `CountVectorizer` and fit them with a `RandomForestClassifier`.

The model learns the relationship between the frequencies of words in the vocabulary, and whether such words indicate a positive or negative sentiment.

In [10]:
%%time
clf = RandomForestClassifier(n_estimators=100)
clf.fit(train_features, df_train['sentiment'])

Wall time: 1min 19s


## Testing
Load and clean the test data similarly, before making predictions with the trained classifier.

In [11]:
df_test = pd.read_csv(join(src, 'testData.tsv'), sep='\t')
df_test

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...
5,2913_8,"...as valuable as King Tut's tomb! (OK, maybe ..."
6,4396_1,This has to be one of the biggest misfires eve...
7,395_2,"This is one of those movies I watched, and won..."
8,10616_1,The worst movie i've seen in years (and i've s...
9,9074_9,"Five medical students (Kevin Bacon, David Labr..."


In [12]:
df_test['review_cleaned'] = [clean_sentence(s) for s in tqdm(df_test['review'])]
df_test

100%|██████████████████████████████████████████████████████████████████████████| 25000/25000 [00:22<00:00, 1105.25it/s]


,id,review,review_cleaned
0,12311_10,Naturally in a film who's main themes are of m...,naturally film main theme mortality nostalgia ...
1,8348_2,This movie is a disaster within a disaster fil...,movie disaster within disaster film full great...
2,5828_4,"All in all, this is a movie for kids. We saw i...",movie kid saw tonight child loved one point ki...
3,7186_2,Afraid of the Dark left me with the impression...,afraid dark left impression several different ...
4,12128_7,A very accurate depiction of small time mob li...,accurate depiction small time mob life filmed ...
5,2913_8,"...as valuable as King Tut's tomb! (OK, maybe ...",valuable king tut tomb ok maybe valuable worth...
6,4396_1,This has to be one of the biggest misfires eve...,one biggest misfire ever script nice could end...
7,395_2,"This is one of those movies I watched, and won...",one movie watched wondered watch find interest...
8,10616_1,The worst movie i've seen in years (and i've s...,worst movie seen year seen lot movie acting te...
9,9074_9,"Five medical students (Kevin Bacon, David Labr...",five medical student kevin bacon david labracc...


In [14]:
test_features = vectorizer.fit_transform(df_test['review_cleaned'])
test_features = test_features.toarray()
test_features.shape

(25000, 5000)

In [15]:
%%time
pred = clf.predict(test_features)

Wall time: 2.95 s


In [16]:
output = pd.DataFrame({'id': df_test['id'], 'sentiment': pred})
output

,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,0
3,7186_2,0
4,12128_7,1
5,2913_8,0
6,4396_1,1
7,395_2,1
8,10616_1,0
9,9074_9,0


In [18]:
output.to_csv('submission/wordcount_randomforest.csv', index=False)